# 중고나라 휴대폰 거래가격 예측하기
예제에서 사용하는 데이터는 https://www.data.go.kr/에서 제공하는 '네이버 중고나라'데이터, 그리고 통신 3사(SK Telecom. LG U+, kt olleh) 사이트에서 제공하는 휴대폰 공시가격 데이터를 기반으로 만들어진 것입니다.

## Step 1 탐색적 분석: 중고나라 데이터 분석하기
예제에서 사용할 중고나라 데이터셋은 다음과 같은 피처로 구성되어 있습니다.
- create_date: 판매(혹은 구매) 게시글이 올라온 시점
- price: 게시글 작성자가 제안한 휴대폰의 거래가격
- text: 게시글의 제목과 본문을 합친 텍스트 데이터
- phone_model: 휴대폰의 기종
- factory_price: 휴대폰 공시가격
- maker: 휴대폰 제조사
- price_index: 판매 게시글이 올라온 시점에서의 휴대폰 물가 지수 데이터

아래의 코드를 통해 데이터를 살펴본 결과, 약 5000여 개의 데이터가 결측값 없이 구성되어 있다는 것을 알 수 있습니다.
#### 중고나라 데이터셋 살펴보기

In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('data/used_mobile_phone.csv')
print(df.info())
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/used_mobile_phone.csv'

다음으로 개별 피처들을 탐색합니다.
#### 개별 피처 탐색하기: data 피처 탐색

In [2]:
#  create_date로 부터 '월'을 의미하는 mnth 정보를 피처로 추출합니다.
df['month'] = df['create_date'].apply(lambda x: x[:7])

# 월별 거래 횟수를 계산하여 출력합니다.
df['month'].value_counts()

NameError: name 'df' is not defined